<hr>
<h1 align="center"><b>Spatiotemporal Graph Neural Networks with <img src="https://raw.githubusercontent.com/TorchSpatiotemporal/tsl/main/docs/source/_static/img/tsl_logo.svg" width="40px" align="center" style="display: inline-block"/> tsl</b></h2>
<hr>

<a href="https://colab.research.google.com/github/TorchSpatiotemporal/tsl/blob/main/examples/notebooks/a_gentle_introduction_to_tsl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this tutorial notebook, we will see how to train our custom-made **Spatiotemporal Graph Neural Network (STGNN) for traffic forecasting** using <img src="https://raw.githubusercontent.com/TorchSpatiotemporal/tsl/main/docs/source/_static/img/tsl_logo.svg" width="30px" align="center" style="display: inline-block"/> [**tsl (Torch Spatiotemporal)**](https://torch-spatiotemporal.readthedocs.io/), a Python library for **neural spatiotemporal data processing**, with a focus on Graph Neural Networks.

It is built upon the most used libraries of the **python scientific computing ecosystem**, with the final objective of providing a straightforward process that goes from data preprocessing to model prototyping.

In particular, tsl offers a wide range of utilities to develop neural networks in [PyTorch](https://pytorch.org/) and [PyTorch Geometric (PyG)](https://www.pyg.org/) for processing **spatiotemporal graph signals**.

<hr>

# **Quickstart**

<hr>

## Installation
<hr>

Let's start by installing tsl from source and the related dependencies. Installing tsl from GitHub ensures to be up-to-date with the latest version.

In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install -q git+https://github.com/TorchSpatiotemporal/tsl.git@dev

We refer to [tsl](https://torch-spatiotemporal.readthedocs.io/en/latest/usage/quickstart.html) and [PyG](https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html) installation guidelines for the setup in other environments.

Let's check if everything is ok.

In [2]:
import tsl
import torch
import numpy as np
import pandas as pd

print(f"tsl version  : {tsl.__version__}")
print(f"torch version: {torch.__version__}")

pd.options.display.float_format = '{:.2f}'.format
np.set_printoptions(edgeitems=3, precision=3)
torch.set_printoptions(edgeitems=2, precision=3)

# Utility functions ################
def print_matrix(matrix):
    return pd.DataFrame(matrix)

def print_model_size(model):
    tot = sum([p.numel() for p in model.parameters() if p.requires_grad])
    out = f"Number of model ({model.__class__.__name__}) parameters:{tot:10d}"
    print("=" * len(out))
    print(out)

tsl version  : 0.9.0
torch version: 1.10.0


## Usage
<hr>

tsl is more than a collection of layers. We can classify the library modules into:

* **Data loading modules**<br>
  Manage how to store, load, and preprocess spatiotemporal data, providing a simple interface to make data ready for downstream neural models.

* **Inference modules**<br>
   Models and engines that take as input spatiotemporal data to make inferences for the task at hand, e.g., forecasting or imputation.

We will go deeper on them in next sections.

<hr>

# **Loading and Preprocessing Data**

<hr>

## Loading a tabular dataset
<hr>

`tsl` comes with several datasets used in spatiotemporal processing literature. You can find them inside the submodule [`tsl.datasets`](https://torch-spatiotemporal.readthedocs.io/en/latest/modules/datasets.html).

As an example, we start by using the [MetrLA](https://paperswithcode.com/sota/traffic-prediction-on-metr-la) dataset, a common benchmark for traffic forecasting. The dataset contains traffic readings collected from 207 loop detectors on highways in Los Angeles County, aggregated in 5 minute intervals for 4 months between March 2012 to June 2012. Loading the dataset is as simple as that:

In [3]:
from tsl.datasets import MetrLA

dataset = MetrLA(root='./data')

print(dataset)

download: 14.5MB [00:04, 3.23MB/s]                            


MetrLA(length=34272, n_nodes=207, n_channels=1)


All the datasets in tsl are subclass of the root class [`tsl.datasets.Dataset`](https://torch-spatiotemporal.readthedocs.io/en/latest/modules/datasets_prototypes.html#tsl.datasets.prototypes.Dataset), exposing useful APIs for spatiotemporal datasets. We can see that data are organized a 3-dimensional array, with:

* **34.272** time steps (1 each 5 minute for 4 months)
* **207** nodes (the loop detectors)
* **1** channels (detected speed)

Nice! Other than storing the data of interest, the dataset comes with useful tools.

In [4]:
print(f"Sampling period: {dataset.freq}")
print(f"Has missing values: {dataset.has_mask}")
print(f"Percentage of missing values: {(1 - dataset.mask.mean()) * 100:.2f}%")
print(f"Has exogenous variables: {dataset.has_covariates}")
print(f"Covariates: {', '.join(dataset.covariates.keys())}")

Sampling period: <5 * Minutes>
Has missing values: True
Percentage of missing values: 8.11%
Has exogenous variables: True
Covariates: dist


Let's look at the output. We know that the dataset has missing entries, with `dataset.mask` being a binary indicator associated with each timestep, node and channel (with ones indicating valid values).

Also, the dataset has a **covariate** attribute (i.e., exogenous variables) – the distance matrix – containing the pairwise distances between sensors.

You can access covariates by `dataset.{covariate_name}`:

In [6]:
print_matrix(dataset.dist)

,0,1,2,3,4,5,6,7,8,9,...,197,198,199,200,201,202,203,204,205,206
0,0.00,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,3764.70,inf,inf,9204.30,inf,inf,8114.80,10009.70
1,inf,0.00,2504.60,8563.80,8572.50,9561.00,9590.00,2506.30,inf,inf,...,inf,inf,inf,inf,4941.90,7559.20,7877.20,inf,inf,inf
2,inf,1489.30,0.00,6971.30,6978.30,9148.20,9177.10,3995.70,inf,inf,...,inf,9467.80,inf,inf,6431.40,7821.80,9366.60,inf,inf,9837.00
3,inf,6805.90,9293.60,0.00,1745.50,6068.80,5401.50,9312.30,inf,inf,...,inf,5906.50,inf,inf,inf,inf,inf,inf,inf,7604.40
4,inf,6606.70,9111.30,1767.40,0.00,4464.00,3655.90,9113.10,inf,inf,...,inf,7207.10,inf,inf,inf,inf,inf,inf,inf,8905.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,inf,inf,10987.10,inf,inf,inf,inf,10133.60,inf,inf,...,inf,inf,inf,inf,9098.90,0.00,4072.70,inf,inf,inf
203,inf,10723.20,9461.80,inf,inf,inf,inf,8608.30,inf,inf,...,inf,inf,inf,inf,7283.50,inf,0.00,inf,inf,inf
204,inf,inf,inf,inf,inf,inf,inf,inf,9189.80,3171.10,...,3672.40,inf,inf,inf,inf,inf,inf,0.00,inf,inf
205,9599.80,inf,inf,inf,inf,inf,inf,inf,inf,10167.50,...,inf,inf,10500.80,inf,inf,inf,inf,inf,0.00,inf


This matrix stores the pairwise distance between sensors, with `inf` denoting two non-neighboring sensors.

Let's now check how the speed readings look like.

In [7]:
dataset.dataframe()

nodes,773869,767541,767542,717447,717446,717445,773062,767620,737529,717816,...,772167,769372,774204,769806,717590,717592,717595,772168,718141,769373
channels,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2012-03-01 00:00:00,64.38,67.62,67.12,61.50,66.88,68.75,65.12,67.12,59.62,62.75,...,45.62,65.50,64.50,66.43,66.88,59.38,69.00,59.25,69.00,61.88
2012-03-01 00:05:00,62.67,68.56,65.44,62.44,64.44,68.11,65.00,65.00,57.44,63.33,...,50.67,69.88,66.67,58.56,62.00,61.11,64.44,55.89,68.44,62.88
2012-03-01 00:10:00,64.00,63.75,60.00,59.00,66.50,66.25,64.50,64.25,63.88,65.38,...,44.12,69.00,56.50,59.25,68.12,62.50,65.62,61.38,69.86,62.00
2012-03-01 00:15:00,64.00,63.75,60.00,59.00,66.50,66.25,64.50,64.25,63.88,65.38,...,44.12,69.00,56.50,59.25,68.12,62.50,65.62,61.38,69.86,62.00
2012-03-01 00:20:00,64.00,63.75,60.00,59.00,66.50,66.25,64.50,64.25,63.88,65.38,...,44.12,69.00,56.50,59.25,68.12,62.50,65.62,61.38,69.86,62.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-06-27 23:35:00,65.00,65.89,68.56,61.67,32.83,54.56,62.44,63.33,59.22,65.33,...,52.89,69.00,65.11,55.67,66.33,62.44,66.78,64.89,69.67,62.33
2012-06-27 23:40:00,61.38,65.62,66.50,62.75,32.83,50.50,62.00,67.00,65.25,67.12,...,54.00,69.25,60.12,60.50,67.25,59.38,66.00,61.25,69.00,62.00
2012-06-27 23:45:00,67.00,59.67,69.56,61.00,32.83,44.78,64.22,63.78,59.78,57.67,...,51.33,67.89,64.33,57.00,66.00,62.67,68.67,63.33,67.44,61.22


### Connecting sensors

Besides the time series, to properly use graph-based models, we need to __connect__ nodes somehow.

With the method [`dataset.get_similarity()`](https://torch-spatiotemporal.readthedocs.io/en/latest/modules/datasets_prototypes.html#tsl.datasets.prototypes.Dataset.get_similarity) we can retrieve nodes' similarities computed with different methods. The available similarity methods for a dataset can be found at `dataset.similarity_options`, while the default one is at `dataset.similarity_score`.

In [8]:
print(f"Default similarity: {dataset.similarity_score}")
print(f"Available similarity options: {dataset.similarity_options}")
print("==========================================")

sim = dataset.get_similarity("distance")  # or dataset.compute_similarity()

print("Similarity matrix W:")
print_matrix(sim)

Default similarity: distance
Available similarity options: {'distance'}
Similarity matrix W:


,0,1,2,3,4,5,6,7,8,9,...,197,198,199,200,201,202,203,204,205,206
0,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.12,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,1.00,0.39,0.00,0.00,0.00,0.00,0.39,0.00,0.00,...,0.00,0.00,0.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00
2,0.00,0.72,1.00,0.00,0.00,0.00,0.00,0.09,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,1.00,0.63,0.00,0.01,0.00,0.00,0.00,...,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.63,1.00,0.05,0.14,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,1.00,0.08,0.00,0.00,0.00
203,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00
204,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.22,...,0.13,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
205,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00


With this method, we compute weight $w_t^{i,j}$ of the edge connecting $i$-th and $j$-th node as<br>
$$
w^{i,j} = \left\{\begin{array}{cl}
     \exp \left(-\frac{\operatorname{dist}\left(i, j\right)^{2}}{\gamma}\right) & \operatorname{dist}\left(i, j\right) \leq \delta  \\
     0 & \text{otherwise}
\end{array}\right. ,
$$<br>
where $\operatorname{dist}\left(i, j\right)$ is the distance between $i$-th and $j$-th node, $\gamma$ controls the kernel width and $\delta$ is a threshold. Notice that in this case the similarity matrix is not symmetric, since the original preprocessed distance matrix is not symmetric too.

So far so good, now we can build an adjacency matrix out ouf the computed similarity.

The method [`dataset.get_connectivity()`](https://torch-spatiotemporal.readthedocs.io/en/latest/modules/datasets_prototypes.html#tsl.datasets.prototypes.Dataset.get_connectivity) – calling `dataset.get_similarity()` under-the-hood – provides useful preprocessing options, and, eventually, returns a possibly sparse, possibly weighted, adjacency matrix.

In [9]:
connectivity = dataset.get_connectivity(threshold=0.1,
                                        include_self=False,
                                        normalize_axis=1,
                                        layout="edge_index")

Let's see what happens with this function call:

1. compute the similarity matrix as before;
1. set to 0 values **below** 0.1 (`threshold=0.1`);
1. **remove** self loops (`include_self=False`);
1. **normalize** edge weights by the **in degree** of nodes (`normalize_axis=1`);
1. request the sparse **COO layout** of PyG (`layout="edge_index"`)

The connectivity matrix with `edge_index` layout is provided in COO format, adopting the convention and notation used in PyTorch Geometric. The returned connectivity is a tuple (`edge_index`, `edge_weight`), where `edge_index` lists all edges as pairs of source-target nodes (dimensions `[2, E]`) and `edge_weight` (dimension `[E]`) stores the corresponding weights.

In [10]:
edge_index, edge_weight = connectivity

print(f'edge_index {edge_index.shape}:\n', edge_index)
print(f'edge_weight {edge_weight.shape}:\n', edge_weight)

edge_index (2, 1515):
 [[  0   0   0 ... 206 206 206]
 [ 13  36  58 ... 157 162 198]]
edge_weight (1515,):
 [0.08  0.163 0.031 ... 0.045 0.031 0.115]


The `"dense"` layout instead corresponds to the weighted adjacency matrix $A \in \mathbb{R}^{N \times N}$. The module [`tsl.ops.connectivity`](https://torch-spatiotemporal.readthedocs.io/en/latest/modules/ops.html#module-tsl.ops.connectivity) contains useful operations for connectivities, including methods to change layout.

In [11]:
from tsl.ops.connectivity import edge_index_to_adj

adj = edge_index_to_adj(edge_index, edge_weight)
print(f'A {adj.shape}:')
print_matrix(adj)

A (207, 207):


,0,1,2,3,4,5,6,7,8,9,...,197,198,199,200,201,202,203,204,205,206
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,0.00,0.14,0.00,0.00,0.00,0.00,0.14,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.19,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,0.12,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.11,0.00,0.00,0.02,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
203,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
204,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.09,...,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
205,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


From the dense layout, the sparse COO format can be easily retrieved as:

In [12]:
print(f'Sparse edge weights:\n', adj[edge_index[1], edge_index[0]])

Sparse edge weights:
 [0.08  0.163 0.031 ... 0.045 0.031 0.115]


## Building a PyTorch-ready dataset
<hr>

In this section, we will see how to fetch **spatiotemporal graph signals** that are then given as input to a neural network (e.g., an STGNN) starting from a dataset of this kind.

The first class that comes in help is [`tsl.data.SpatioTemporalDataset`](https://torch-spatiotemporal.readthedocs.io/en/latest/modules/data_pytorch_datasets.html#tsl.data.SpatioTemporalDataset). This class is a subclass of [`torch.utils.data.Dataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) and is in charge of mapping a tabular dataset represented in your preferred format (e.g., numpy array, pandas dataframe or the aforementioned [`tsl.datasets.Dataset`](https://torch-spatiotemporal.readthedocs.io/en/latest/modules/datasets_prototypes.html#tsl.datasets.prototypes.Dataset)) to a PyTorch-ready implementation.

In particular, a `SpatioTemporalDataset` object can be used to achieve the following:
* Perform **data manipulation** operations required to feed the data to a PyTorch module (e.g., casting data to `torch.tensor`, handling possibly different `shapes`, synchronizing temporal data).
* Create **`(input, target)` samples** for supervised learning following the [**sliding window**](https://torch-spatiotemporal.readthedocs.io/en/latest/usage/spatiotemporal_dataset.html#sliding-window) approach.
* Define how data should be **arranged** in a **spatiotemporal graph signal** (e.g.,  which are the inputs and targets, how node attributes and covariates variables are mapped into a single graph).
* **Preprocess** data before creating a **spatiotemporal graph signal** by appling [**transformations**](https://torch-spatiotemporal.readthedocs.io/en/latest/modules/transforms.html) or [**scaling**](https://torch-spatiotemporal.readthedocs.io/en/latest/modules/data_preprocessing.html) operations.

Let's see how to go from a `Dataset` to a `SpatioTemporalDataset`.

In [13]:
from tsl.data import SpatioTemporalDataset

torch_dataset = SpatioTemporalDataset(target=dataset.dataframe(),
                                      connectivity=connectivity,
                                      mask=dataset.mask,
                                      horizon=12,
                                      window=12,
                                      stride=1)
print(torch_dataset)

SpatioTemporalDataset(n_samples=34249, n_nodes=207, n_channels=1)


As you can see, the number of samples is not the same as the number of steps we have in the dataset. Indeed, we divided the historic time series with a **sliding window** of **12 time steps** for the **lockback window** (`window=12`), with a corresponding **horizon** of **12 time steps** (`horizon=12`). Thus, a single sample spans for a total of 24 time steps. The `stride` parameters set how many time steps intercurring between two subsequent samples. The following picture helps at visualizing how these (and more) parameters affect the slicing of the original time series in samples.

<p align="center">
  <img src="https://torch-spatiotemporal.readthedocs.io/en/latest/_images/sliding_window.svg" width="500em">
</p>

## Spatiotemporal graph signals in tsl

We now have a PyTorch-based dataset containing a collection of spatiotemporal graph signals. We can fetch samples in the same way we fetch elements of a Python list. Let's look in details to the layout of a sample:

In [14]:
sample = torch_dataset[0]
print(sample)

Data(
  input=(x=[t=12, n=207, f=1], edge_index=[2, e=1515], edge_weight=[e=1515]),
  target=(y=[t=12, n=207, f=1]),
  has_mask=True
)


A sample is of type [`tsl.data.Data`](https://torch-spatiotemporal.readthedocs.io/en/latest/modules/data_objects.html#tsl.data.Data), the base class for representing spatiotemporal graph signals in tsl. This class extends [`torch_geometric.data.Data`](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.data.Data.html#torch_geometric.data.Data), preserving all its functionalities and
adding utilities for spatiotemporal data processing. The main APIs of `Data` include:

* **`Data.input`**: view on the tensors stored in `Data` that are meant to serve as input to the model.
  In the simplest case of a single node-attribute matrix, we could just have `Data.input.x`.
* **`Data.target`**: view on the tensors stored in `Data` used as labels to train the model.
  In the common case of a single label, we could just have `Data.input.y`.
* **`Data.edge_index`**: graph connectivity in COO format (i.e., as node pairs).
* **`Data.edge_weight`**: weights of the graph connectivity, if any.
* **`Data.mask`**: binary mask indicating the data in `Data.target.y` to be used as ground-truth for the loss (default is `None`).
* **`Data.transform`**: mapping of `ScalerModule`, whose keys must be
  transformable (or transformed) tensors in `Data`.
* **`Data.pattern`**: mapping containing the *pattern* for each tensor in `Data`. Patterns add information about the dimensions of tensors (e.g., specifying which are the time step and node dimensions).

None of these attributes are required and custom attributes can be seamlessly added.

Let's check more in details how each of these attributes is composed.

#### Input and Target

`Data.input` and `Data.target` provide a **view** on the unique (shared) storage in `Data`, such that the same key in `Data.input` and `Data.target` cannot reference different objects.

In [15]:
sample.input.to_dict()

{'x': tensor([[[64.375],
          [67.625],
          ...,
          [69.000],
          [61.875]],
 
         [[62.667],
          [68.556],
          ...,
          [68.444],
          [62.875]],
 
         ...,
 
         [[65.222],
          [63.667],
          ...,
          [67.222],
          [60.000]],
 
         [[62.250],
          [67.750],
          ...,
          [66.500],
          [59.429]]]),
 'edge_index': tensor([[  0,   0,  ..., 206, 206],
         [ 13,  36,  ..., 162, 198]]),
 'edge_weight': tensor([0.080, 0.163,  ..., 0.031, 0.115])}

In [16]:
sample.target.to_dict()

{'y': tensor([[[61.125],
          [67.000],
          ...,
          [70.000],
          [63.500]],
 
         [[58.556],
          [62.667],
          ...,
          [66.222],
          [62.333]],
 
         ...,
 
         [[63.222],
          [64.111],
          ...,
          [67.333],
          [63.000]],
 
         [[65.000],
          [63.000],
          ...,
          [64.125],
          [60.750]]])}

#### Mask and Transform

`mask` and `transform` are just symbolic links to the corresponding object inside the storage. They also expose properties `has_mask` and `has_transform`.

In [17]:
if sample.has_mask:
    print(sample.mask)
else:
    print("Sample has no mask.")

tensor([[[True],
         [True],
         ...,
         [True],
         [True]],

        [[True],
         [True],
         ...,
         [True],
         [True]],

        ...,

        [[True],
         [True],
         ...,
         [True],
         [True]],

        [[True],
         [True],
         ...,
         [True],
         [True]]])


In [18]:
if sample.has_transform:
    print(sample.transform)
else:
    print("Sample has no transformation functions.")

Sample has no transformation functions.


#### Pattern

The `pattern` mapping can be useful to glimpse on how data are arranged.
The convention we use is the following:

* `'t'` stands for the **time steps** dimension
* `'n'` stands for a **node** dimension
* `'e'` stands for the **edge** dimension
* `'f'` stands for a **feature** dimension
* `'b'` stands for the **batch** dimension


In [19]:
print(sample.pattern)
print("==================   Or we can print patterns and shapes together   ==================")
print(sample)

{'x': 't n f', 'mask': 't n f', 'edge_index': '2 e', 'edge_weight': 'e', 'y': 't n f'}
==================   Or we can print patterns and shapes together   ==================
Data(
  input=(x=[t=12, n=207, f=1], edge_index=[2, e=1515], edge_weight=[e=1515]),
  target=(y=[t=12, n=207, f=1]),
  has_mask=True
)


## Batching spatiotemporal graph signals

Getting a batch of spatiotemporal graph signals from a single dataset is as simple as accessing multiple elements from a list:

In [20]:
batch = torch_dataset[:5]
print(batch)

StaticBatch(
  input=(x=[b=5, t=12, n=207, f=1], edge_index=[2, e=1515], edge_weight=[e=1515]),
  target=(y=[b=5, t=12, n=207, f=1]),
  has_mask=True
)


As you can see, we now have an additional dimension for the time-varying elements (i.e., `x` and `y`) denoted by pattern `b`, i.e., the batch dimension. In this new, first dimension we stacked the features of the first 5 spatiotemporal graphs in the dataset.

Note that this is possible only because we are assuming a fixed underlying topology, as also confirmed by the `edge_index` and `edge_weight` attributes. The explanation on how `Data` objects with different graphs are batched together is out of the scope of this notebook.

## Preparing the dataset for training
<hr>

Usually, before running an experiment there are two quite common preprocessing steps:

* **splitting** the dataset into **training/validation/test** sets;
* **data preprocessing** (scaling/normalizing data, detrending).

In tsl, these operations are managed by the [`tsl.data.SpatioTemporalDataModule`](https://torch-spatiotemporal.readthedocs.io/en/latest/modules/data_datamodule.html#tsl.data.datamodule.SpatioTemporalDataModule), which is based on the [`LightningDataModule`](https://pytorch-lightning.readthedocs.io/en/stable/api/pytorch_lightning.core.LightningDataModule.html#pytorch_lightning.core.LightningDataModule) from [PyTorch Lightning](https://pytorch-lightning.readthedocs.io/en/stable/). A `DataModule` allows us to standardize and make consistent the training, validation, test splits, data preparation and transformations across different environments and experiments.

Let's see an example

In [21]:
from tsl.data.datamodule import (SpatioTemporalDataModule,
                                 TemporalSplitter)
from tsl.data.preprocessing import StandardScaler

# Normalize data using mean and std computed over time and node dimensions
scalers = {'target': StandardScaler(axis=(0, 1))}

# Split data sequentially:
#   |------------ dataset -----------|
#   |--- train ---|- val -|-- test --|
splitter = TemporalSplitter(val_len=0.1, test_len=0.2)

dm = SpatioTemporalDataModule(
    dataset=torch_dataset,
    scalers=scalers,
    splitter=splitter,
    batch_size=64,
)

print(dm)

SpatioTemporalDataModule(train_len=None, val_len=None, test_len=None, scalers=[target], batch_size=64)


You can consider to extend the base `SpatioTemporalDataModule` to add further processing to fit your needs.

At this point, the `DataModule` object has not actually performed any processing yet (lazy approach).

We can execute the preprocessing routines by calling the `dm.setup()` method.

In [22]:
dm.setup()
print(dm)

SpatioTemporalDataModule(train_len=24648, val_len=2728, test_len=6849, scalers=[target], batch_size=64)


During `setup` the datamodule does the following operations:

1. Carries out the dataset splitting into training/validation/test sets according to the provided [`Splitter`](https://torch-spatiotemporal.readthedocs.io/en/latest/modules/data_datamodule.html#tsl.data.datamodule.splitters.Splitter).
1. Fits all the `Scalers` on the training data in `torch_dataset` corresponding to the scalers' key.

### Splitters

Splitters in tsl are the objects defining the policy of data splitting. [Read more](https://torch-spatiotemporal.readthedocs.io/en/latest/modules/data_datamodule.html#splitters)

### Scalers

The `tsl.data.preprocessing` package offers several of the most common data normalization techniques under the `tsl.data.preprocessing.Scaler` interface.
They adopt an API similar to `scikit-learn`'s scalers, with `fit`/`transform`/`fit_transform`/`inverse_transform` methods. [Read more](https://torch-spatiotemporal.readthedocs.io/en/latest/modules/data_preprocessing.html#scalers)

<hr>

# **Building Spatiotemporal Graph Neural Networks**

<hr>

In this section, we will see how to build a very simple  Spatiotemporal Graph Neural Network.

All the functions and classes needed to build neural networks in tsl are under the [`tsl.nn`](https://torch-spatiotemporal.readthedocs.io/en/latest/modules/nn.html) module.


## The `nn` module
<hr>

The [`tsl.nn`](https://torch-spatiotemporal.readthedocs.io/en/latest/modules/nn.html) module is organized as follows:

```
tsl
└── nn
    ├── base
    ├── blocks
    ├── layers
    └── models
```

The 3 most important submodules in it are [`layers`](https://torch-spatiotemporal.readthedocs.io/en/latest/modules/nn_layers.html), [`blocks`](https://torch-spatiotemporal.readthedocs.io/en/latest/modules/nn_blocks.html), and [`models`](https://torch-spatiotemporal.readthedocs.io/en/latest/modules/nn_models.html), ordered by increasing level of abstraction.

### Layers

A **layer** is a basic building block for our neural networks. In simple words, a layer takes an input, performs one (or few) operations, and returns a transformation of the input. Examples of layers are [`DiffConv`](https://torch-spatiotemporal.readthedocs.io/en/latest/modules/nn_layers.html#tsl.nn.layers.graph_convs.DiffConv), which implements the [diffusion convolution](https://arxiv.org/abs/1707.01926) operation, or [`LayerNorm`](https://torch-spatiotemporal.readthedocs.io/en/latest/modules/nn_layers.html#tsl.nn.layers.norm.LayerNorm).

### Blocks

**blocks** perform more complex transformations or combine several operations. We divide blocks into **encoders**, if they provide a representation of the input in a new space, and **decoders**, if they produce a meaningful output from a representation.

### Models

We wrap a series of operations, represented by blocks and/or layers, in a **model**. A model takes as input a spatiotemporal graph signal and returns the desired output, e.g., the forecasted node features at future time steps.

## Designing a custom STGNN
<hr>

Let's get the hands dirty and create our first simple STGNN! We will follow the **Time-then-Space** paradigm. We use a GRU shared among the nodes to process the temporal dimension. This will give us in output a single feature vector for each node, which is then propagated through the underlying graph using a Diffusion Convolutional GNN. Before and after, we add linear transformations to encode the input features and decode the learned representations. We also make use of **node embeddings** (free parameters learned individually for each node) to make our STGNN a **global-local model** ([Cini et al., 2023](https://arxiv.org/abs/2302.04071)).

All the layers that we need are provided inside `tsl.nn`. We use:
* [`RNN`](https://torch-spatiotemporal.readthedocs.io/en/latest/modules/nn_blocks.html#tsl.nn.blocks.encoders.RNN) from `tsl.nn.blocks.encoders` for the GRU;
* [`DiffConv`](https://torch-spatiotemporal.readthedocs.io/en/latest/modules/nn_layers.html#tsl.nn.layers.graph_convs.DiffConv) from `tsl.nn.layers.graph_convs` for the diffusion convolution;
* `StaticGraphEmbedding` from `tsl.nn.base` for the node embeddings.

In [23]:
import torch.nn as nn

from tsl.nn.blocks.encoders import RNN
from tsl.nn.layers import NodeEmbedding, DiffConv
from einops.layers.torch import Rearrange  # reshape data with Einstein notation


class TimeThenSpaceModel(nn.Module):
    def __init__(self, input_size: int, n_nodes: int, horizon: int,
                 hidden_size: int = 32,
                 rnn_layers: int = 1,
                 gnn_kernel: int = 2):
        super(TimeThenSpaceModel, self).__init__()

        self.encoder = nn.Linear(input_size, hidden_size)

        self.node_embeddings = NodeEmbedding(n_nodes, hidden_size)

        self.time_nn = RNN(input_size=hidden_size,
                           hidden_size=hidden_size,
                           n_layers=rnn_layers,
                           cell='gru',
                           return_only_last_state=True)
        
        self.space_nn = DiffConv(in_channels=hidden_size,
                                 out_channels=hidden_size,
                                 k=gnn_kernel)

        self.decoder = nn.Linear(hidden_size, input_size * horizon)
        self.rearrange = Rearrange('b n (t f) -> b t n f', t=horizon)

    def forward(self, x, edge_index, edge_weight):
        # x: [batch time nodes features]
        x_enc = self.encoder(x)  # linear encoder: x_enc = xΘ + b
        x_emb = x_enc + self.node_embeddings()  # add node-identifier embeddings
        h = self.time_nn(x_emb)  # temporal processing: x=[b t n f] -> h=[b n f]
        z = self.space_nn(h, edge_index, edge_weight)  # spatial processing
        x_out = self.decoder(z)  # linear decoder: z=[b n f] -> x_out=[b n t⋅f]
        x_horizon = self.rearrange(x_out)
        return x_horizon

We can play with hyperparameters and make an instance of our model.

In [24]:
hidden_size = 32   #@param
rnn_layers = 1     #@param
gnn_kernel = 2     #@param

input_size = torch_dataset.n_channels   # 1 channel
n_nodes = torch_dataset.n_nodes         # 207 nodes
horizon = torch_dataset.horizon         # 12 time steps

stgnn = TimeThenSpaceModel(input_size=input_size,
                           n_nodes=n_nodes,
                           horizon=horizon,
                           hidden_size=hidden_size,
                           rnn_layers=rnn_layers,
                           gnn_kernel=gnn_kernel)
print(stgnn)
print_model_size(stgnn)

TimeThenSpaceModel(
  (encoder): Linear(in_features=1, out_features=32, bias=True)
  (node_embeddings): NodeEmbedding(n_nodes=207, embedding_size=32)
  (time_nn): RNN(
    (rnn): GRU(32, 32)
  )
  (space_nn): DiffConv(32, 32)
  (decoder): Linear(in_features=32, out_features=12, bias=True)
  (rearrange): Rearrange('b n (t f) -> b t n f', t=12)
)
Number of model (TimeThenSpaceModel) parameters:     18572


Fine, we loaded the data and built a model, so let's train it!

## Setting up training
<hr>

We are now ready to train our model. We set up the training procedure as we prefer, in the following we will use PyTorch Lightning's Trainer to reduce the burder of the dirty work. We recall that tsl is highly integrated with widely used PyTorch-based libraries, such as PyTorch Lightning and PyTorch Geometric.

### The Predictor

In tsl, inference engines are implemented as a [`LightningModule`](https://pytorch-lightning.readthedocs.io/en/latest/api/pytorch_lightning.core.LightningModule.html#pytorch_lightning.core.LightningModule). [`tsl.engines.Predictor`](https://torch-spatiotemporal.readthedocs.io/en/latest/modules/engines.html#tsl.engines.Predictor) is a base class that can be extended to build more complex forecasting approaches.
These modules are meant to wrap deep models in order to ease training and inference phases.

In [25]:
from tsl.metrics.torch import MaskedMAE, MaskedMAPE
from tsl.engines import Predictor

loss_fn = MaskedMAE()

metrics = {'mae': MaskedMAE(),
           'mape': MaskedMAPE(),
           'mae_at_15': MaskedMAE(at=2),  # '2' indicates the third time step,
                                          # which correspond to 15 minutes ahead
           'mae_at_30': MaskedMAE(at=5),
           'mae_at_60': MaskedMAE(at=11)}

# setup predictor
predictor = Predictor(
    model=stgnn,                   # our initialized model
    optim_class=torch.optim.Adam,  # specify optimizer to be used...
    optim_kwargs={'lr': 0.001},    # ...and parameters for its initialization
    loss_fn=loss_fn,               # which loss function to be used
    metrics=metrics                # metrics to be logged during train/val/test
)

Now let's finalize the last details. We make use of [TensorBoard](https://www.tensorflow.org/tensorboard/) to log and visualize metrics.

In [26]:
from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger(save_dir="logs", name="tsl_intro", version=0)

In [27]:
%load_ext tensorboard
%tensorboard --logdir logs

We let [`pytorch_lightning.Trainer`](https://pytorch-lightning.readthedocs.io/en/stable/common/trainer.html) handle the dirty work for us. We can directly pass the datamodule to the trainer for fitting.

If this is the case, the trainer will call the `setup` method, and then load train and validation sets.

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    dirpath='logs',
    save_top_k=1,
    monitor='val_mae',
    mode='min',
)

trainer = pl.Trainer(max_epochs=100,
                     logger=logger,
                     gpus=1 if torch.cuda.is_available() else None,
                     limit_train_batches=100,  # end an epoch after 100 updates
                     callbacks=[checkpoint_callback])

trainer.fit(predictor, datamodule=dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type               | Params
-----------------------------------------------------
0 | loss_fn       | MaskedMAE          | 0     
1 | train_metrics | MetricCollection   | 0     
2 | val_metrics   | MetricCollection   | 0     
3 | test_metrics  | MetricCollection   | 0     
4 | model         | TimeThenSpaceModel | 18.6 K
-----------------------------------------------------
18.6 K    Trainable params
0         Non-trainable params
18.6 K    Total params
0.074     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/Caskroom/miniconda/base/envs/tsl-dev/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Only args ['x', 'edge_index', 'edge_weight'] are forwarded to the model (TimeThenSpaceModel). 
/usr/local/Caskroom/miniconda/base/envs/tsl-dev/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

## Testing
<hr>


Now let's see how the trained model behaves on new unseen data.

In [ ]:
predictor.load_model(checkpoint_callback.best_model_path)
predictor.freeze()

trainer.test(predictor, datamodule=dm);

Cool! We succeeded in creating our first simple – yet effective – Spatiotemporal GNN!

🥷 We are now **tsl ninjas**. We learned how to:

* Load benchmark datasets
* Organize data for processing
* Preprocess the data
* Build a Spatiotemporal GNN
* Train and evaluate models

We hope you enjoyed this introduction to tsl, now go and build the game-changer STGNN out there 🌎

🧡 The tsl team